# Strategies
We can do a value interation (inferior) or policy interation (superior) strategy to find the best action or policy to take.

Value interation to find the optimal policy is very expensive.

In [33]:
import numpy as np
blocked = [(1,1)]
goal = (2,3)
death = (1,3)
def build_env():
    arr = np.negative(np.zeros([3,4]))
    arr[goal] = 100
    arr[death] = -100
    for wall in blocked:
        arr[wall] = None
    arr[0,0] = 2
    return arr
env = build_env()
print (env)

[[   2.   -0.   -0.   -0.]
 [  -0.   nan   -0. -100.]
 [  -0.   -0.   -0.  100.]]


## what is the value function?
two types:
let $ E $ = expected
let $ \pi $ = policy ( the action )
let $ * $ = optimal ( best )
- State-Value Function ( $ V^{*}(s) = R(s) + \gamma{\max_a}Q^{*}(s,a) $ )
    - Value of the state
    - This is the total reward of the system( final value)
- Action-Value Function ( $ Q^{*}(s,a) = \Sigma_{s'}{P(s' \mid s,a)V^{\ast}(s') $ )
    - Value of the action
    - Per state
the reward function is the same:   $ {G}_t = \sum_{k=0}^{\infty} {\gamma ^ k}{R}_{t+1+k} $

bellman's equation is plug in the best action $ Q^* $ solving for the best  $ V^{\ast} = R(s) + \gamma{\max_a} \Sigma_{s'}{P(s' \mid s,a)V^{\ast}(s') $

### Value interation

In [34]:
location = (2, 3)
directions = {(0, 1) : [None, None],
              (1, 0) : [None, None],
              (-1, 0) : [None, None],
              (0, -1) : [None, None]}
val_map = []
gamma = 1
reward = -4
def allowed_states(state):  # get adjacent spaces
    p = []
    for direction in directions.keys():
        r, c = tuple([sum(x) for x in zip(direction, state)])
        r = min(r,2) # hit wall
        c = min(c,3)
        if (r, c) in blocked or (r, c) == state: # hit any obstacles
            r, c = state  # np.multiply((-1, -1), state)
        coordinate = (r, c)
        print (coordinate)
        directions[direction] = coordinate
        if coordinate == death or coordinate == state or coordinate == goal:
            continue
        else:
            p.append((r,c))
    return p


prev = allowed_states(location)
print (env)
print (prev)


(2, 3)
(2, 3)
(1, 3)
(2, 2)
[[   2.   -0.   -0.   -0.]
 [  -0.   nan   -0. -100.]
 [  -0.   -0.   -0.  100.]]
[(2, 2)]


Only one previous state allows us to get here:
we need to calculate the bellman's equation for that state: $ V^{\ast} = R(s) + \gamma{\max_a} \Sigma_{s'}{P(s' \mid s,a)V^{\ast}(s') \quad for~R(s)=0.04 \quad \gamma=1$
for previous states: calculate each action:
- Up: $ 0.4 + (1) * (  ~  (V_{12}*0.8) + (V_{21}*0.1) + (V_{23}*0.1)    ~   ) \quad \forall V = 0, ~ V_{23}=100   $
- Down: $ 0.4 + (1) * (  ~  (V_{22}*0.8) + (V_{11}*0.1) + (V_{23}*0.1)    ~   ) \quad \forall V = 0, ~ V_{23}=100   $
- Left $ 0.4 + (1) * (  ~  (V_{11}*0.8) + (V_{12}*0.1) + (V_{22}*0.1)    ~   ) \quad \forall V = 0, ~ V_{23}=100   $
- Right: $ 0.4 + (1) * (  ~  (V_{23}*0.8) + (V_{22}*0.1) + (V_{12}*0.1)    ~   ) \quad \forall V = 0, ~ V_{23}=100   $



In [35]:
def bellman(vp, vs, vss):
    vpi = gamma*(  (vp*0.8) +  (vs*0.1)  + (vss*0.1)  )
    return vpi

def calculate(states):
    env_val = {}
    for state in states:
        expected_reward=0
        allowed_states(state)
        for direction in directions:  #  find the two adjacent sides geographic location
            vp = directions[direction]
            if direction[0] != 0:  # Vertical movement
                vs = directions [(0, 1)]
                vss = directions [(0, -1)]
            else:  # lateral movement
                 vs = directions [(1, 0)]
                 vss = directions [(-1, 0)]

            vp = env[vp]  # expected_reward
            vs = env[vs]
            vss = env[vss]

            expected_reward = max(bellman(vp,vs,vss), expected_reward) # math
        env_val[state] = reward + expected_reward
    return env_val

g = calculate(prev)
print(g)


(2, 3)
(2, 2)
(1, 2)
(2, 1)
{(2, 2): 76.0}
